In [1]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, AutoTokenizer
from datasets import Dataset,DatasetDict
from transformers import default_data_collator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd
import json

In [42]:
qa_herbert_model = AutoModelForQuestionAnswering.from_pretrained("allegro/herbert-base-cased")
# qa_cased_bert_model = AutoModelForQuestionAnswering.from_pretrained("dkleczek/bert-base-polish-cased-v1")
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized

In [43]:
def to_squad_format(dataset):
    res = {
        "id": [],
        "question": [],
        "answers": [],
        "context": [],
        "title": []
    }
    for act in dataset["data"]:
        title = act["title"]
        for para in act["paragraphs"]:
            ctx = para["context"]
            for q in para["qas"]:
                q["context"] = ctx
                q["title"] = title
                answers = {"answer_start": [], "text": []}
                for an in q["answers"]:
                    answers["answer_start"].append(an["answer_start"])
                    answers["text"].append(an["text"])
                q["answers"] = answers
                for c in ["id", "question", "answers", "context", "title"]:
                    res[c].append(q[c])
    return Dataset.from_dict(res)
    
def read_dataset():
    dataset = {}
    for t in ["train", "dev", "test"]:
        with open(f"lqad-pl-1/lqad-pl-{t}.json") as f:
            dataset[t] = to_squad_format(json.loads(f.read()))
    return DatasetDict(dataset)

dataset = read_dataset()


In [44]:
dataset["train"][0]

{'id': '2007_adw_1',
 'question': 'Wybrany do Sejmu może być obywatel polski mający prawo wybierania, który: ',
 'answers': {'answer_start': [115],
  'text': ['najpóźniej w dniu wyborów kończy 21 lat']},
 'context': ' Rozdział IV SEJM I SENAT Art . 99 § 1 . Wybrany do Sejmu może być obywatel polski mający prawo wybierania , który najpóźniej w dniu wyborów kończy 21 lat . § 2 . Wybrany do Senatu może być obywatel polski mający prawo wybierania , który najpóźniej w dniu wyborów kończy 30 lat . § 3 . Wybraną do Sejmu lub do Senatu nie może być osoba skazana prawomocnym wyrokiem na karę pozbawienia wolności za przestępstwo umyślne ścigane z oskarżenia publicznego .',
 'title': 'Konstytucja Rzeczypospolitej Polskiej z dnia 2 kwietnia 1997 r. uchwalona przez Zgromadzenie Narodowe w dniu 2 kwietnia 1997 r., przyjęta przez Naród w referendum konstytucyjnym w dniu 25 maja 1997 r., podpisana przez Prezydenta Rzeczypospolitej Polskiej w dniu 16 lipca 1997 r'}

In [45]:
# src: https://huggingface.co/docs/transformers/custom_datasets#load-squad-dataset
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [46]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [48]:
def finetune(model):
    data_collator = default_data_collator

    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["dev"],
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

    trainer.train()
    return model

In [49]:
# qa_herbert_model_tuned = finetune(qa_herbert_model)
# qa_herbert_model_tuned.save_pretrained("herbert-base-cased-legal-qa-tuned")

qa_herbert_model_tuned = AutoModelForQuestionAnswering.from_pretrained("herbert-base-cased-legal-qa-tuned", local_files_only=True)

In [9]:
# qa_cased_bert_model_tuned = finetune(qa_cased_bert_model)
# qa_cased_bert_model_tuned.save_pretrained("bert-base-polish-cased-v1-legal-qa-tuned")

qa_cased_bert_model_tuned_t = AutoModelForQuestionAnswering.from_pretrained("bert-base-polish-cased-v1-legal-qa-tuned", local_files_only=True)

In [50]:
def evaluate_transformers(model):
    trainer = Trainer(model=model,
                      tokenizer=tokenizer)

    return trainer.predict(tokenized_dataset["test"])

In [51]:
raw_predictions = evaluate_transformers(qa_herbert_model_tuned)

***** Running Prediction *****
  Num examples = 463
  Batch size = 8


In [56]:
from tqdm.auto import tqdm
import collections


pad_on_right = tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
squad_v2 = False

def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [57]:
validation_features = dataset["dev"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset["dev"].column_names
)
final_predictions = postprocess_qa_predictions(dataset["dev"], validation_features, raw_predictions.predictions)

  0%|          | 0/1 [00:00<?, ?ba/s]

Post-processing 444 example predictions split into 445 features.


  0%|          | 0/444 [00:00<?, ?it/s]

In [58]:
final_predictions

OrderedDict([('2007_adw_54', '. Zakaz prowadzenia'),
             ('2007_adw_61', 'lub środek karny nie podlega'),
             ('2007_adw_62', 'niewykonania kary nie popełni'),
             ('2007_adw_64', 'zakaz prowadzenia pojazdów'),
             ('2009_adw_rad_37', ''),
             ('2009_adw_rad_38', 'Orzeczona'),
             ('2013_adw_rad_29', '44 § 3 .'),
             ('2016_adw_rad_24', '. Odpowiedzialność'),
             ('2016_adw_rad_25', 'czynu'),
             ('2017_adw_rad_25', 'przeszkodzie orzeczeniu'),
             ('2017_adw_rad_26',
              '. Karalność wykroczenia ustaje , jeżeli od czasu jego popełnienia upłynął rok ; jeżeli w'),
             ('2019_adw_rad_17', 'za popełnione w czasie ,'),
             ('2019_adw_rad_18', 'określonych w niniejszej ustawie'),
             ('2019_adw_rad_19', 'karnych na podstawie innych naruszonych'),
             ('2019_adw_rad_20', 'L'),
             ('2019_adw_rad_21', 'miejsca'),
             ('2019_adw_rad_22', 'któr

In [77]:
for d in dataset["dev"]:
    print(d["question"])
    print(d["answers"]["text"][0])
    print(final_predictions[d["id"]])
    print("\n")

Zgodnie z Kodeksem wykroczeń, niedopuszczalne jest orzeczenie zakazu prowadzenia pojazdów w rozmiarze: 
3
. Zakaz prowadzenia


Zgodnie z Kodeksem wykroczeń, środek karny nie podlega wykonaniu, jeżeli od daty: 
uprawomocnienia się rozstrzygnięcia upłynęły 3 lata
lub środek karny nie podlega


Zgodnie z Kodeksem wykroczeń, można warunkowo zawiesić wykonanie kary: 
aresztu
niewykonania kary nie popełni


Kodeks wykroczeń nie przewiduje środka karnego w postaci: 
publicznej
zakaz prowadzenia pojazdów


Zgodnie z Kodeksem wykroczeń, kara aresztu trwa: 
najkrócej 5 , najdłużej 30 dni



Zgodnie z Kodeksem wykroczeń, orzeczona kara lub środek karny nie podlega wykonaniu, jeżeli od daty uprawomocnienia się rozstrzygnięcia: 
upłynęły 3 lata
Orzeczona


 Zgodnie z Kodeksem wykroczeń, w przypadku orzeczenia kary aresztu z warunkowym zawieszeniem jej wykonania, ukaranie uważa się za niebyłe:  
Jeżeli w okresie próby i w ciągu dalszych 2 miesięcy nie zarządzono wykonania kary
44 § 3 .


 Zgodnie z

 Zgodnie z ustawą – Prawo o postępowaniu przed sądami administracyjnymi, sąd może zawiesić postępowanie z urzędu: 
jeżeli ujawni się czyn , którego ustalenie w drodze karnej lub dyscyplinarnej mogłoby wywrzeć wpływ na rozstrzygnięcie sprawy sądowoadministracyjnej
) jeżeli


Zgodnie z ustawą - Prawo o postępowaniu przed sądami administracyjnymi, osobie prawnej, a także innej jednostce organizacyjnej nieposiadającej osobowości prawnej, może być przyznane prawo pomocy w zakresie całkowitym, gdy:  
wykaże , że nie ma żadnych środków na poniesienie jakichkolwiek kosztów postępowania
wykaże , że nie ma żadnych


 Zgodnie z ustawą – Prawo o postępowaniu przed sądami administracyjnymi, od orzeczeń Naczelnego Sądu 
nie przysługuje , z wyjątkiem gdy niezgodność z prawem wynika z rażącego naruszenia norm prawa Unii Europejskiej
orzeczenia Art . 28


Zgodnie z ustawą - Prawo o postępowaniu przed sądami administracyjnymi, w razie wniesienia skargi do sądu po wszczęciu postępowania administracyjnego

In [69]:
"2019_adw_rad_26" in final_predictions

False